In [34]:
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interactive, FloatSlider
from collections import Counter

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, log_loss, roc_curve, auc
from sklearn import svm

from xgboost import XGBClassifier, plot_importance

from imblearn.over_sampling import RandomOverSampler


In [2]:
def scores(ytest, predict):
    print("Recall:  " + str(recall_score(ytest, predict)))
    print("Accuracy:  " + str(accuracy_score(ytest, predict)))
    print()
    print(confusion_matrix(ytest, predict))

In [3]:
with open('sj_df.pickle', 'rb') as read_file:
    sj_df = pickle.load(read_file)
    
with open('iq_df.pickle', 'rb') as read_file:
    iq_df = pickle.load(read_file)

In [4]:
sj_df['isFallorWinter'] = np.where(sj_df['weekofyear'] >= 35, 1, 0)

In [5]:
iq_df['q*Td'] = iq_df['reanalysis_specific_humidity_g_per_kg'] * iq_df['reanalysis_dew_point_temp_k']

In [6]:
sj_df.head()

,outbreak,city,year,weekofyear,total_cases,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,isFallorWinter
0,0,sj,1990,18,4,1990-04-30,0.122600,0.103725,0.198483,0.177617,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,0
1,0,sj,1990,19,5,1990-05-07,0.169900,0.142175,0.162357,0.155486,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,0
2,0,sj,1990,20,4,1990-05-14,0.032250,0.172967,0.157200,0.170843,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,0
3,0,sj,1990,21,3,1990-05-21,0.128633,0.245067,0.227557,0.235886,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,0
4,0,sj,1990,22,6,1990-05-28,0.196200,0.262200,0.251200,0.247340,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,0


In [7]:
sj_df.outbreak.value_counts()

0    495
1    441
Name: outbreak, dtype: int64

In [8]:
iq_df.head()

,outbreak,city,year,weekofyear,total_cases,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,q*Td
1008,0,iq,2001,47,1,2001-11-19,0.329700,0.330417,0.340643,0.441571,...,88.837143,65.89,18.451429,9.228571,27.575000,11.575000,33.7,20.8,31.0,5475.514147
1009,0,iq,2001,48,1,2001-11-26,0.301500,0.361300,0.305571,0.369917,...,87.377143,47.46,17.427143,8.014286,28.233333,11.666667,35.3,21.8,12.2,5155.148024
1010,0,iq,2001,49,1,2001-12-03,0.325057,0.226471,0.299257,0.350471,...,87.240000,46.86,17.595714,8.871429,28.425000,11.650000,35.5,21.2,23.1,5208.004651
1011,0,iq,2001,50,2,2001-12-10,0.326033,0.235533,0.315571,0.306643,...,96.118571,138.22,19.305714,7.785714,27.840000,10.960000,35.0,21.0,75.0,5743.450000
1012,0,iq,2001,51,4,2001-12-17,0.222943,0.224071,0.212814,0.184129,...,93.947143,43.95,18.965714,7.385714,27.075000,9.500000,32.9,21.0,57.9,5636.827037


In [9]:
iq_df.outbreak.value_counts()

0    376
1     72
Name: outbreak, dtype: int64

In [10]:
mm_scaler = MinMaxScaler()
sj_train, sj_validate, sj_test = np.split(sj_df.sample(frac=1), [int(.6*len(sj_df)), int(.8*len(sj_df))])

sj_X_train = sj_train.iloc[:,6:]
sj_y_train = sj_train.iloc[:,0]

sj_X_val = sj_validate.iloc[:,6:]
sj_y_val = sj_validate.iloc[:,0]

sj_X_test = sj_test.iloc[:,6:]
sj_y_test = sj_test.iloc[:,0]

sj_X_train_mm = mm_scaler.fit_transform(sj_X_train)
sj_X_val_mm = mm_scaler.fit_transform(sj_X_val)
sj_X_test_mm = mm_scaler.fit_transform(sj_X_test)

iq_train, iq_validate, iq_test = np.split(iq_df.sample(frac=1), [int(.6*len(iq_df)), int(.8*len(iq_df))])

iq_X_train = iq_train.iloc[:,6:]
iq_y_train = iq_train.iloc[:,0]

iq_X_val = iq_validate.iloc[:,6:]
iq_y_val = iq_validate.iloc[:,0]

iq_X_test = iq_test.iloc[:,6:]
iq_y_test = iq_test.iloc[:,0]

iq_X_train_mm = mm_scaler.fit_transform(iq_X_train)
iq_X_val_mm = mm_scaler.fit_transform(iq_X_val)
iq_X_test_mm = mm_scaler.fit_transform(iq_X_test)

In [11]:
sj_train.head()

,outbreak,city,year,weekofyear,total_cases,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,isFallorWinter
240,1,sj,1994,49,221,1994-12-10,0.206684,0.050855,0.191304,0.194407,...,85.570000,45.95,16.580000,1.500000,26.371429,6.185714,30.0,22.2,58.2,1
334,1,sj,1996,40,30,1996-09-30,-0.038500,0.068133,0.165129,0.195214,...,79.177143,27.58,17.521429,2.100000,27.414286,5.885714,31.1,23.9,20.9,1
853,1,sj,2006,38,33,2006-09-24,-0.055600,-0.016200,0.197700,0.171929,...,75.861429,27.18,18.210000,3.414286,28.828571,6.800000,33.3,23.9,2.3,1
349,1,sj,1997,3,33,1997-01-15,-0.145900,0.033800,0.223371,0.217086,...,74.940000,0.00,14.278571,2.242857,24.385714,6.657143,28.9,19.4,0.6,0
768,0,sj,2005,5,8,2005-02-05,0.044200,-0.075800,0.205600,0.178143,...,78.640000,0.00,13.770000,1.971429,23.714286,4.528571,27.8,19.4,5.4,0


In [12]:
logit_sj = LogisticRegression(C=100, penalty='l2')
logit_sj_fit = logit_sj.fit(sj_X_train_mm, sj_y_train)
logit_sj_predict = logit_sj_fit.predict(sj_X_val_mm)

/Users/joelporcaro/anaconda3/envs/metis/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
print(np.mean(cross_val_score(logit_sj, sj_X_train_mm, sj_y_train, cv=kf)))

0.7130372945638432


/Users/joelporcaro/anaconda3/envs/metis/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/joelporcaro/anaconda3/envs/metis/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/joelporcaro/anaconda3/envs/metis/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/joelporcaro/anaconda3/envs/metis/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/joelporcaro/anaconda3/envs/metis/lib/python3.6/site-packages/sklearn/

In [14]:
scores(sj_y_val, logit_sj_predict)

Recall:  0.4875
Accuracy:  0.6363636363636364

[[80 27]
 [41 39]]


In [15]:
def make_confusion_matrix(model, X_test, y_test, threshold=0.5):
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (model.predict(X_test) does this automatically with a threshold of 0.5)
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    fraud_confusion = confusion_matrix(y_test, y_predict)
    plt.figure(dpi=80)
    sns.heatmap(fraud_confusion, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['healthy', 'outbreak'],
           yticklabels=['healthy', 'outbreak']);
    plt.xlabel('prediction')
    plt.ylabel('actual')

In [16]:
interactive(lambda threshold: make_confusion_matrix(logit_sj, sj_X_val_mm, sj_y_val, threshold), threshold=(0.0,1.0,0.02))

interactive(children=(FloatSlider(value=0.5, description='threshold', max=1.0, step=0.02), Output()), _dom_cla…

In [17]:
sj_predictions = logit_sj_fit.predict_proba(sj_X_test_mm)[:,1]
sj_predict_final = np.where(sj_predictions > 0.3, 1, 0)

In [65]:
scores(sj_y_test, sj_predict_final)

Recall:  0.8369565217391305
Accuracy:  0.6648936170212766

[[48 48]
 [15 77]]


In [19]:
iq_X_train.reset_index().head()

,index,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,q*Td
0,1080,0.205243,0.184200,0.193129,0.208786,78.590,298.082857,298.971429,297.358571,305.9,...,96.270000,78.590,19.078571,7.028571,28.175000,9.675000,34.50,22.90,79.00,5673.176745
1,1069,0.224957,0.260271,0.206929,0.289500,84.050,298.125714,299.564286,295.001429,307.7,...,84.460000,84.050,16.514286,10.271429,28.400000,10.600000,34.20,23.00,67.10,4871.737878
2,1195,0.291557,0.246071,0.351857,0.260457,50.880,297.857143,299.057143,294.644286,307.7,...,84.467143,50.880,16.168571,10.485714,27.350000,9.550000,32.80,22.40,89.20,4763.977180
3,1039,0.179171,0.279100,0.132614,0.213286,3.180,295.808571,296.907143,292.147143,305.3,...,82.167143,3.180,13.870000,10.728571,26.150000,10.800000,32.60,18.60,7.10,4052.080871
4,1118,0.202571,0.198967,0.244557,0.187957,63.975,296.867143,297.896429,296.160000,303.3,...,96.365714,63.975,17.714286,5.192857,27.366667,9.133333,33.65,22.35,161.45,5246.262857


In [42]:
ros = RandomOverSampler(random_state=0)
iq_X_train_mm_resampled, iq_y_train_resampled = ros.fit_sample(iq_X_train_mm, iq_y_train)
iq_X_val_mm_resampled, iq_y_val_resampled = ros.fit_sample(iq_X_val_mm, iq_y_val)

In [43]:
Counter(iq_y_val_resampled)

Counter({0: 73, 1: 73})

In [44]:
rf_iq = RandomForestClassifier(n_estimators=400, min_samples_split=10, min_samples_leaf=4, 
                               max_features='auto', max_depth=70, bootstrap=True)
rf_iq_fit = rf_iq.fit(iq_X_train_mm_resampled, iq_y_train_resampled)
rf_iq_predict = rf_iq_fit.predict(iq_X_val_mm_resampled)

In [45]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
print(np.mean(cross_val_score(rf_iq, iq_X_train_mm_resampled, iq_y_train_resampled, cv=kf)))

0.9497611084567608


In [46]:
scores(iq_y_val_resampled, rf_iq_predict)

Recall:  0.0
Accuracy:  0.4794520547945205

[[70  3]
 [73  0]]


In [47]:
interactive(lambda threshold: make_confusion_matrix(rf_iq, iq_X_val_mm_resampled, iq_y_val_resampled, threshold), threshold=(0.0,1.0,0.02))

interactive(children=(FloatSlider(value=0.5, description='threshold', max=1.0, step=0.02), Output()), _dom_cla…

In [53]:
iq_predictions = rf_iq_fit.predict_proba(iq_X_test_mm)[:,1]
iq_predict_final = np.where(iq_predictions > 0.2, 1, 0)

In [54]:
scores(iq_y_test, iq_predict_final)

Recall:  0.5625
Accuracy:  0.5666666666666667

[[42 32]
 [ 7  9]]


In [57]:
rf_iq = DecisionTreeClassifier()
rf_iq_fit = rf_iq.fit(iq_X_train_mm_resampled, iq_y_train_resampled)
rf_iq_predict = rf_iq_fit.predict(iq_X_val_mm_resampled)

In [58]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
print(np.mean(cross_val_score(rf_iq, iq_X_train_mm_resampled, iq_y_train_resampled, cv=kf)))

0.9126612517916864


In [59]:
scores(iq_y_val_resampled, rf_iq_predict)

Recall:  0.0958904109589041
Accuracy:  0.5068493150684932

[[67  6]
 [66  7]]


In [64]:
coefs = []
for i in rf_iq_fit.feature_importances_:
    coefs.append(i)
    
labels = []
for i in iq_X_train.columns:
    labels.append(i)
    
for label, coef in zip(labels, coefs):
    print(str(label) + "  " + str(coef))

ndvi_ne  0.0
ndvi_nw  0.020042692545024103
ndvi_se  0.0
ndvi_sw  0.22617420269433203
precipitation_amt_mm  0.029112081513828235
reanalysis_air_temp_k  0.030335165088071193
reanalysis_avg_temp_k  0.011644832605531294
reanalysis_dew_point_temp_k  0.0
reanalysis_max_air_temp_k  0.005109467367733115
reanalysis_min_air_temp_k  0.019606707158301342
reanalysis_precip_amt_kg_per_m2  0.1838375511608738
reanalysis_relative_humidity_percent  0.08474089672115118
reanalysis_sat_precip_amt_mm  0.06539287799528672
reanalysis_specific_humidity_g_per_kg  0.007874520987434281
reanalysis_tdtr_k  0.0
station_avg_temp_c  0.06570652782443175
station_diur_temp_rng_c  0.0821302369801652
station_max_temp_c  0.036848098319183974
station_min_temp_c  0.05766636727080266
station_precip_mm  0.07377777376784922
q*Td  0.0


In [26]:
rf = RandomForestClassifier()
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [27]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [28]:
rf_random.fit(iq_X_train_mm, iq_y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   54.4s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None